In [36]:
!pip install numpy==1.24.4 catboost==1.2
!pip install kneed

  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\joao altarugio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "C:\Users\joao altarugio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\joao altarugio\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 137, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "C:\Users\joao altarugio\AppData\Lo

In [37]:
!pip install xgboost

In [38]:
!pip install lightgbm

In [39]:
!pip install ngboost

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score



from catboost import CatBoostRegressor
from kneed import KneeLocator

# Inicializando os dados

In [41]:
df = pd.read_csv('/Users/joao altarugio/Desktop/Projeto LaMav/data/Really important data/RefractiveIndex_clean.csv')
df

,H,Li,Be,B,C,N,O,F,Na,Mg,...,Re,Pt,Au,Hg,Tl,Pb,Bi,Th,U,RefractiveIndex
0,0.0,0.0,0.0,0.0,0.0,0.0,0.630000,0.0,0.073333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.657
1,0.0,0.0,0.0,0.0,0.0,0.0,0.624881,0.0,0.070428,0.019714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.651
2,0.0,0.0,0.0,0.0,0.0,0.0,0.620012,0.0,0.067698,0.038418,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.648
3,0.0,0.0,0.0,0.0,0.0,0.0,0.615143,0.0,0.064971,0.057114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.644
4,0.0,0.0,0.0,0.0,0.0,0.0,0.622678,0.0,0.069171,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47949,0.0,0.0,0.0,0.0,0.0,0.0,0.614246,0.0,0.068289,0.009847,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.503
47950,0.0,0.0,0.0,0.0,0.0,0.0,0.610767,0.0,0.080119,0.006285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.500
47951,0.0,0.0,0.0,0.0,0.0,0.0,0.613590,0.0,0.078927,0.006285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.495
47952,0.0,0.0,0.0,0.0,0.0,0.0,0.610737,0.0,0.080132,0.006350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.500


In [42]:
df = df.loc[:, (df != 0).any(axis=0)]

In [43]:
df.describe()

,Li,Be,B,O,Na,Mg,Al,Si,P,K,...,Hf,Ta,W,Hg,Tl,Pb,Bi,Th,U,RefractiveIndex
count,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,...,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000,47954.000000
mean,0.015972,0.000509,0.076831,0.606466,0.027071,0.004443,0.014806,0.093528,0.026567,0.012645,...,0.000045,0.002057,0.001686,0.000023,0.000606,0.009223,0.005789,0.000291,0.000009,1.692723
std,0.040336,0.005764,0.100164,0.033962,0.050295,0.016148,0.032890,0.097050,0.064510,0.033314,...,0.000992,0.008816,0.008012,0.001353,0.009210,0.033196,0.030441,0.003222,0.000586,0.185597
min,0.000000,0.000000,0.000000,0.376053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.400000
25%,0.000000,0.000000,0.000000,0.586667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.547000
50%,0.000000,0.000000,0.016504,0.603990,0.000000,0.000000,0.000000,0.044976,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.640000
75%,0.000000,0.000000,0.139873,0.624222,0.038462,0.000000,0.011591,0.188315,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.802000
max,0.559868,0.182826,0.400000,0.739130,0.553169,0.304328,0.379653,0.353035,0.285714,0.417960,...,0.049850,0.228547,0.222222,0.135722,0.405002,0.436782,0.374397,0.140000,0.065133,2.750000


In [44]:
df_metrics = pd.DataFrame()

# Separando os Folds


In [45]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
target_column = "RefractiveIndex"

folds = []

for train_idx, val_idx in kf.split(df):
    # Separar dados de treino e validação
    train_df = df.iloc[train_idx].copy()
    val_df = df.iloc[val_idx].copy()

    # Separar features e alvo
    X_train = train_df.drop(columns=target_column)
    y_train = train_df[target_column]
    X_val = val_df.drop(columns=target_column)
    y_val = val_df[target_column]

    # Normalização MinMax apenas com treino
    scaler = MinMaxScaler()
    scaler.fit(X_train)

    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Reconstruir DataFrames normalizados (com os nomes originais das colunas)
    train_processed = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    train_processed[target_column] = y_train

    val_processed = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
    val_processed[target_column] = y_val

    folds.append((train_processed, val_processed))

# Exemplo: mostrar formas do primeiro fold
print("Fold 0 - treino:", folds[0][0].shape, "| validação:", folds[0][1].shape)


Fold 0 - treino: (43158, 64) | validação: (4796, 64)


# Regressao para montar curva

In [46]:
def obter_resultados_folds(folds, target_column):
    resultados_por_iteracao = {}

    for test_fold_idx in range(len(folds)):
        # Fold atual para teste externo
        test_df = folds[test_fold_idx][1]

        # Demais folds para treino/validação internos
        inner_folds = [folds[i] for i in range(len(folds)) if i != test_fold_idx]

        resultados_folds_internos = []

        for i, (train_df, val_df) in enumerate(inner_folds):
            X_train = train_df.drop(columns=target_column)
            y_train = train_df[target_column]
            X_val = val_df.drop(columns=target_column)
            y_val = val_df[target_column]

            # Modelos
            knn = KNeighborsRegressor(n_neighbors=5)
            rf = RandomForestRegressor(n_estimators=390, max_features='sqrt', random_state=42)
            cart = DecisionTreeRegressor(criterion='friedman_mse', min_impurity_decrease=0.0012, random_state=42)
            svr = SVR(kernel='rbf')
            etr = ExtraTreesRegressor(n_estimators=390, max_features='sqrt', random_state=42)

            # Treinamento
            knn.fit(X_train, y_train)
            rf.fit(X_train, y_train)
            cart.fit(X_train, y_train)
            svr.fit(X_train, y_train)
            etr.fit(X_train, y_train)

            # Previsões no fold interno
            partial_df = pd.DataFrame({
                "RefractiveIndex": y_val,
                "Previsao_KNN": knn.predict(X_val),
                "Previsao_RF": rf.predict(X_val),
                "Previsao_CART": cart.predict(X_val),
                "Previsao_SVR": svr.predict(X_val),
                "Previsao_ExtraTrees": etr.predict(X_val)
            }, index=y_val.index)

            resultados_folds_internos.append(partial_df)

        # Junta os resultados de todos os folds internos nesta iteração
        df_metrics = pd.concat(resultados_folds_internos).sort_index()
        chave_iteracao = f"iteracao_fold_{test_fold_idx}"
        resultados_por_iteracao[chave_iteracao] = {
            "df_metrics": df_metrics,
            "test_df": test_df  #guarda o conjunto de teste usado na iteração
        }

    return resultados_por_iteracao


In [47]:
resultados = obter_resultados_folds(folds, target_column='RefractiveIndex')

# Exemplo para acessar e imprimir resultados da iteração 0
print("Resultados da iteração 0:")
print(resultados["iteracao_fold_0"]["df_metrics"])

Resultados da iteração 0:
       RefractiveIndex  Previsao_KNN  Previsao_RF  Previsao_CART  \
0                1.657        1.6538     1.654348       1.657735   
1                1.651        1.6582     1.651608       1.648667   
2                1.648        1.6590     1.655144       1.649667   
3                1.644        1.6570     1.660408       1.698500   
5                1.683        1.6878     1.677889       1.655000   
...                ...           ...          ...            ...   
47949            1.503        1.4982     1.502061       1.518496   
47950            1.500        1.4992     1.501818       1.492980   
47951            1.495        1.5004     1.502569       1.508214   
47952            1.500        1.4974     1.502523       1.508722   
47953            1.503        1.4988     1.502962       1.507091   

       Previsao_SVR  Previsao_ExtraTrees  
0          1.629415             1.654500  
1          1.640260             1.658000  
2          1.652395         

In [48]:
#Caminho onde você quer salvar — altere se quiser salvar em outro local
caminho_arquivo = "/Users/joao altarugio/Desktop/Projeto LaMav/data/resultados_todas_colunas_final.pkl"

#Salvando o dicionário em arquivo
with open(caminho_arquivo, "wb") as f:
    pickle.dump(resultados, f)

print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

Arquivo salvo com sucesso em: /Users/joao altarugio/Desktop/Projeto LaMav/data/resultados_todas_colunas_final.pkl


In [ ]:
caminho_arquivo = "/Users/joao altarugio/Desktop/Projeto LaMav/data/resultados_todas_colunas_final.pkl"
with open(caminho_arquivo, "rb") as f:
   resultados = pickle.load(f)

In [50]:
for chave in resultados:
    df_manipulacao = resultados[chave]["df_metrics"]
    if "Previsao_SVR" in df_manipulacao.columns:
        resultados[chave]["df_metrics"] = df_manipulacao.drop(columns=["Previsao_SVR"])
print(resultados["iteracao_fold_0"]["df_metrics"])


       RefractiveIndex  Previsao_KNN  Previsao_RF  Previsao_CART  \
0                1.657        1.6538     1.654348       1.657735   
1                1.651        1.6582     1.651608       1.648667   
2                1.648        1.6590     1.655144       1.649667   
3                1.644        1.6570     1.660408       1.698500   
5                1.683        1.6878     1.677889       1.655000   
...                ...           ...          ...            ...   
47949            1.503        1.4982     1.502061       1.518496   
47950            1.500        1.4992     1.501818       1.492980   
47951            1.495        1.5004     1.502569       1.508214   
47952            1.500        1.4974     1.502523       1.508722   
47953            1.503        1.4988     1.502962       1.507091   

       Previsao_ExtraTrees  
0                 1.654500  
1                 1.658000  
2                 1.654669  
3                 1.656346  
5                 1.677603  
...      

In [51]:
print("\nConjunto de teste externo (test_df) da iteração 0:")
print(resultados["iteracao_fold_0"]["test_df"])


Conjunto de teste externo (test_df) da iteração 0:
             Li   Be         B         O        Na   Mg        Al        Si  \
4      0.000000  0.0  0.000000  0.702195  0.125045  0.0  0.000000  0.097966   
38     0.000000  0.0  0.583846  0.673187  0.055152  0.0  0.000000  0.043208   
49     0.000000  0.0  0.000000  0.723043  0.132569  0.0  0.000000  0.103861   
71     0.000000  0.0  0.000000  0.786416  0.097540  0.0  0.000000  0.076417   
75     0.000000  0.0  0.000000  0.712238  0.128641  0.0  0.000000  0.100784   
...         ...  ...       ...       ...       ...  ...       ...       ...   
47911  0.000000  0.0  0.323351  0.650215  0.000000  0.0  0.000000  0.136904   
47921  0.010496  0.0  0.606815  0.617293  0.000000  0.0  0.006241  0.008994   
47923  0.000000  0.0  0.622325  0.628947  0.000000  0.0  0.000000  0.002223   
47932  0.000000  0.0  0.613849  0.621905  0.000000  0.0  0.000000  0.003000   
47933  0.005456  0.0  0.623077  0.632201  0.000000  0.0  0.000000  0.005294   


## Criando dados para segundo modelo identificando maiores erros curva

In [52]:
def encontrar_maiores_erros_media(df_metrics):
    # Calcula a média das previsões dos modelos
    previsoes_modelos = df_metrics[[
        "Previsao_KNN",
        "Previsao_RF",
        "Previsao_CART",
        "Previsao_ExtraTrees"
    ]]
    media_previsoes = previsoes_modelos.mean(axis=1)

    # Calcula o erro absoluto entre o valor real e a média das previsões
    erro_absoluto_medio = abs(df_metrics["RefractiveIndex"] - media_previsoes)

    # Cria DataFrame com erro e índice original
    df_erros_ordenado = pd.DataFrame({
        "Erro_Absoluto_Medio": erro_absoluto_medio
    })
    df_erros_ordenado['index_real'] = df_erros_ordenado.index  # salva os índices reais
    df_erros_ordenado = df_erros_ordenado.sort_values("Erro_Absoluto_Medio").reset_index(drop=True)

    # Aplica o método do cotovelo
    erro_total = df_erros_ordenado["Erro_Absoluto_Medio"].values
    indices = np.arange(len(erro_total))
    knee = KneeLocator(indices, erro_total, curve="convex", direction="increasing", S=1)

    # Valor do erro no ponto de corte
    erro_corte = erro_total[knee.knee] if knee.knee is not None else None

    # Índices reais das amostras com maiores erros
    array_index_maiores_erros = df_erros_ordenado.loc[knee.knee:, 'index_real'].to_numpy()

    return array_index_maiores_erros, erro_corte


In [53]:
dicionario_indices_maiores_erros = {}
dicionario_erros_de_corte = {}

for chave_iteracao, dados in resultados.items():
    df_metrics = dados["df_metrics"]
    indices_maiores_erros, erro_de_corte = encontrar_maiores_erros_media(df_metrics)

    dicionario_indices_maiores_erros[f"{chave_iteracao}"] = indices_maiores_erros
    dicionario_erros_de_corte[f"{chave_iteracao}"] = erro_de_corte


In [54]:
len(dicionario_indices_maiores_erros['iteracao_fold_0'])

456

In [55]:
dicionario_erros_de_corte['iteracao_fold_0']

0.114280228205482

## Função Edgar

In [56]:
dicionario_df_resultado = {}

for chave_iteracao, dados in resultados.items():
    # Extrai o número do fold
    n = int(chave_iteracao.split("_")[-1])

    # DataFrame com métricas (inclui valor real e predições)
    df_metrics = dados["df_metrics"].copy()

    # DataFrame com features do fold correspondente
    df_features = folds[n][0].copy()

    # Calcula a média das colunas de predição
    col_predicoes = ['Previsao_KNN', 'Previsao_RF', 'Previsao_CART', 'Previsao_ExtraTrees']
    df_metrics['Previsao'] = df_metrics[col_predicoes].mean(axis=1)

    # Mantém apenas a coluna real e a média das previsões
    df_metrics = df_metrics[['RefractiveIndex', 'Previsao']]

    # Remove a coluna target das features (se existir)
    df_features_sem_target = df_features.drop(columns=['RefractiveIndex'], errors='ignore')

    # Concatena valor real, previsão média e features
    df_resultado = pd.concat([
        df_metrics.reset_index(drop=True),
        df_features_sem_target.reset_index(drop=True)
    ], axis=1)

    # Salva no dicionário final
    dicionario_df_resultado[f"{chave_iteracao}"] = df_resultado


In [57]:
def elementos_com_aparicoes_menores(df, limite=208):
    # Considera as colunas de elementos (todas exceto 'RefractiveIndex' e 'Previsao')
    elementos = [col for col in df.columns if col not in ['RefractiveIndex']]
    
    # Contar amostras onde o elemento aparece (valor > 0)
    contagem_aparicoes = {elem: (df[elem] > 0).sum() for elem in elementos}
    
    # Selecionar elementos com aparições menores que o limite
    elementos_filtrados = [elem for elem, contagem in contagem_aparicoes.items() if contagem < limite]
    
    return elementos_filtrados

# Exemplo de uso:
elementos_pouco_frequentes = elementos_com_aparicoes_menores(df, limite=208)
print(elementos_pouco_frequentes)


['Sc', 'Cr', 'Co', 'Ni', 'Cu', 'Se', 'Mo', 'Ru', 'Ag', 'Pr', 'Eu', 'Tb', 'Dy', 'Ho', 'Tm', 'Lu', 'Hg', 'U']


In [58]:
dicionario_df_resultado['iteracao_fold_0']

,RefractiveIndex,Previsao,Li,Be,B,O,Na,Mg,Al,Si,...,Lu,Hf,Ta,W,Hg,Tl,Pb,Bi,Th,U
0,1.657,1.655096,0.0,0.0,0.0,0.723043,0.132569,0.000000,0.000000,0.103861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.651,1.654119,0.0,0.0,0.0,0.708468,0.127318,0.064780,0.000000,0.099747,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.648,1.654620,0.0,0.0,0.0,0.694605,0.122382,0.126238,0.000000,0.095880,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.644,1.668063,0.0,0.0,0.0,0.680743,0.117452,0.187671,0.000000,0.092017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.683,1.674573,0.0,0.0,0.0,0.681700,0.117710,0.000000,0.000000,0.092219,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43153,1.503,1.505204,0.0,0.0,0.0,0.678188,0.123450,0.032356,0.168082,0.653560,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43154,1.500,1.498761,0.0,0.0,0.0,0.668283,0.144837,0.020652,0.167221,0.651071,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43155,1.495,1.503220,0.0,0.0,0.0,0.676321,0.142681,0.020653,0.164568,0.641292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43156,1.500,1.502381,0.0,0.0,0.0,0.668198,0.144860,0.020864,0.166913,0.650815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
from scipy.ndimage import uniform_filter1d
import numpy as np

def calcular_indices_acima_da_curva(
    df_resultado, 
    indices_maiores_erros,  # permanece como argumento por compatibilidade
    erro_de_corte, 
    elementos_pouco_frequentes=None,
    window=200
):
    df_ordenado = df_resultado.sort_values('RefractiveIndex').copy()
    erro = np.abs(df_ordenado['RefractiveIndex'] - df_ordenado['Previsao'])
    erro_suave = uniform_filter1d(erro, size=window)
    deslocamento = erro_de_corte - np.min(erro_suave)
    erro_suave_deslocado = erro_suave + deslocamento

    if elementos_pouco_frequentes is None:
        elementos_pouco_frequentes = []

    def tem_elemento_pouco_frequente(linha):
        return any((el in df_ordenado.columns and getattr(linha, el) > 0) for el in elementos_pouco_frequentes)

    elemento_marcado_x = 39076
    indices_acima_da_curva = []

    for i, linha in enumerate(df_ordenado.itertuples()):
        idx = df_ordenado.index[i]
        if tem_elemento_pouco_frequente(linha):
            continue
        elif erro.iloc[i] > erro_suave_deslocado[i]:
            if idx != elemento_marcado_x:
                indices_acima_da_curva.append(idx)

    # Adiciona manualmente os índices que sempre devem ser incluídos
    indices_fixos = [28252, 29601, 29602, 36935, 40092, 40093, 40094, 41246, 44537]
    for idx in indices_fixos:
        if idx not in indices_acima_da_curva:
            indices_acima_da_curva.append(idx)

    return indices_acima_da_curva


In [60]:
# Supondo que todos os dicionários têm as mesmas chaves do tipo: 'iteracao_fold_0', 'iteracao_fold_1', ...
indices_acima = {}

for chave in dicionario_df_resultado:
    df_resultado = dicionario_df_resultado[chave]
    indices_maiores_erros = dicionario_indices_maiores_erros.get(chave, [])
    erro_de_corte = dicionario_erros_de_corte[chave]

    resultado = calcular_indices_acima_da_curva(
        df_resultado,
        indices_maiores_erros,
        erro_de_corte,
        elementos_pouco_frequentes=elementos_pouco_frequentes
    )

    indices_acima[chave] = resultado


In [61]:
len(indices_acima['iteracao_fold_0'])

293

# Organizando os dados dos dois modelos

In [62]:
def gerar_indices_modelos(df_metrics, indices_maiores_erros):
    array_index_modelo_1 = df_metrics.index.to_numpy()
    array_index_modelo_2 = np.setdiff1d(array_index_modelo_1, indices_maiores_erros)
    return array_index_modelo_1, array_index_modelo_2

In [63]:
dicionario_indices_modelos = {}

for chave_iteracao in resultados.keys():
    df_metrics = resultados[chave_iteracao]["df_metrics"]
    indices_maiores_erros = indices_acima[f"{chave_iteracao}"]

    array_index_modelo_1, array_index_modelo_2 = gerar_indices_modelos(df_metrics, indices_maiores_erros)

    dicionario_indices_modelos[chave_iteracao] = {
        "array_index_modelo_1": array_index_modelo_1,
        "array_index_modelo_2": array_index_modelo_2
    }

# Exemplo de visualização:
for chave, arrays in dicionario_indices_modelos.items():
    print(f"{chave}:\n  Modelo 1: {arrays['array_index_modelo_1']}\n  Modelo 2: {arrays['array_index_modelo_2']}\n")


iteracao_fold_0:
  Modelo 1: [    0     1     2 ... 47951 47952 47953]
  Modelo 2: [    0     1     2 ... 47951 47952 47953]

iteracao_fold_1:
  Modelo 1: [    0     2     3 ... 47951 47952 47953]
  Modelo 2: [    0     2     3 ... 47951 47952 47953]

iteracao_fold_2:
  Modelo 1: [    1     2     3 ... 47951 47952 47953]
  Modelo 2: [    1     2     3 ... 47951 47952 47953]

iteracao_fold_3:
  Modelo 1: [    0     1     2 ... 47951 47952 47953]
  Modelo 2: [    0     1     2 ... 47951 47952 47953]

iteracao_fold_4:
  Modelo 1: [    0     1     2 ... 47949 47951 47953]
  Modelo 2: [    0     1     2 ... 47949 47951 47953]

iteracao_fold_5:
  Modelo 1: [    0     1     2 ... 47950 47951 47952]
  Modelo 2: [    0     1     2 ... 47950 47951 47952]

iteracao_fold_6:
  Modelo 1: [    0     1     2 ... 47951 47952 47953]
  Modelo 2: [    0     1     2 ... 47951 47952 47953]

iteracao_fold_7:
  Modelo 1: [    0     1     2 ... 47951 47952 47953]
  Modelo 2: [    0     1     2 ... 47951 47952 

# A partir daqui começa a ficar diferente

In [64]:
resultados['iteracao_fold_0']['df_metrics']

,RefractiveIndex,Previsao_KNN,Previsao_RF,Previsao_CART,Previsao_ExtraTrees
0,1.657,1.6538,1.654348,1.657735,1.654500
1,1.651,1.6582,1.651608,1.648667,1.658000
2,1.648,1.6590,1.655144,1.649667,1.654669
3,1.644,1.6570,1.660408,1.698500,1.656346
5,1.683,1.6878,1.677889,1.655000,1.677603
...,...,...,...,...,...
47949,1.503,1.4982,1.502061,1.518496,1.502062
47950,1.500,1.4992,1.501818,1.492980,1.501047
47951,1.495,1.5004,1.502569,1.508214,1.501695
47952,1.500,1.4974,1.502523,1.508722,1.500878


In [65]:
def avaliar_modelos(df, array_index_modelo_1, array_index_modelo_2, test_df):
    def treinar_e_prever(df_treino, df_teste):
        X_train = df_treino.drop(columns=['RefractiveIndex'])
        y_train = df_treino['RefractiveIndex']
        X_test = df_teste.drop(columns=['RefractiveIndex'])
        y_test = df_teste['RefractiveIndex']

        scaler = MinMaxScaler()
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

        models = {
            "KNN": KNeighborsRegressor(n_neighbors=5),
            "RandomForest": RandomForestRegressor(n_estimators=390, max_features='sqrt', random_state=42, n_jobs=1),
            "DecisionTree": DecisionTreeRegressor(criterion='friedman_mse', min_impurity_decrease=0.0012, random_state=42),
            "ExtraTrees": ExtraTreesRegressor(n_estimators=390, max_features='sqrt', random_state=42, n_jobs=1)
        }

        predicoes = {}
        for model_name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)

            predicoes[model_name] = y_pred

        return pd.DataFrame({
            'RefractiveIndex_Real': y_test
        }, index=y_test.index).assign(**predicoes)

    df_model_1 = df.loc[df.index.isin(array_index_modelo_1)]
    test_indices = test_df.index
    df_test = df.loc[df.index.isin(test_indices)]

    predicoes_model_1 = treinar_e_prever(df_model_1, df_test)

    df_model_2 = df.loc[df.index.isin(array_index_modelo_2)]
    predicoes_model_2 = treinar_e_prever(df_model_2, df_test)

    return predicoes_model_1, predicoes_model_2


In [66]:
# Dicionário final para armazenar os resultados
resultados_finais = {}

# Itera por todas as iterações dos folds
for chave_fold in resultados.keys():
    # Recupera o test_df da iteração atual
    test_df = resultados[chave_fold]["test_df"]

    # Recupera os arrays de índices dos dois modelos
    array_index_modelo_1 = dicionario_indices_modelos[chave_fold]["array_index_modelo_1"]
    array_index_modelo_2 = dicionario_indices_modelos[chave_fold]["array_index_modelo_2"]

    # Avalia os modelos usando o df original
    resultados_modelo_1, resultados_modelo_2 = avaliar_modelos(
        df, array_index_modelo_1, array_index_modelo_2, test_df
    )

    # Armazena os resultados no dicionário final
    resultados_finais[chave_fold] = {
        "modelo_1": resultados_modelo_1,
        "modelo_2": resultados_modelo_2
    }

# Exemplo de visualização dos resultados:
for chave, resultados_modelos in resultados_finais.items():
    print(f"\n{chave} - Resultados Modelo 1:")
    print(resultados_modelos["modelo_1"].head())  # Mostra as primeiras linhas

    print(f"\n{chave} - Resultados Modelo 2:")
    print(resultados_modelos["modelo_2"].head())  # Mostra as primeiras linhas



iteracao_fold_0 - Resultados Modelo 1:
    RefractiveIndex_Real     KNN  RandomForest  DecisionTree  ExtraTrees
4                  1.676  1.6696      1.675179      1.686800    1.675826
38                 1.536  1.5456      1.599161      1.555579    1.578576
49                 1.658  1.6546      1.654527      1.657870    1.654600
71                 1.779  1.7512      1.787940      1.782400    1.785885
75                 1.685  1.6772      1.681787      1.660286    1.682074

iteracao_fold_0 - Resultados Modelo 2:
    RefractiveIndex_Real     KNN  RandomForest  DecisionTree  ExtraTrees
4                  1.676  1.6696      1.674854      1.686800    1.675728
38                 1.536  1.5456      1.603266      1.555579    1.570229
49                 1.658  1.6546      1.654448      1.657870    1.654600
71                 1.779  1.7512      1.803670      1.782400    1.781164
75                 1.685  1.6772      1.680146      1.660286    1.682738

iteracao_fold_1 - Resultados Modelo 1:
    

## Salvando os dicionarios criados

In [67]:
#Caminho onde você quer salvar — altere se quiser salvar em outro local
caminho_arquivo = "/Users/joao altarugio/Desktop/Projeto LaMav/data/resultados_teste_final.pkl"

#Salvando o dicionário em arquivo
with open(caminho_arquivo, "wb") as f:
    pickle.dump(resultados_finais, f)

print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

Arquivo salvo com sucesso em: /Users/joao altarugio/Desktop/Projeto LaMav/data/resultados_teste_final.pkl


In [68]:
caminho_arquivo = "/Users/joao altarugio/Desktop/Projeto LaMav/data/resultados_teste_final.pkl"

with open(caminho_arquivo, "rb") as f:
   resultados_finais = pickle.load(f)

In [69]:
resultados_finais['iteracao_fold_0']['modelo_1']

,RefractiveIndex_Real,KNN,RandomForest,DecisionTree,ExtraTrees
4,1.676,1.6696,1.675179,1.686800,1.675826
38,1.536,1.5456,1.599161,1.555579,1.578576
49,1.658,1.6546,1.654527,1.657870,1.654600
71,1.779,1.7512,1.787940,1.782400,1.785885
75,1.685,1.6772,1.681787,1.660286,1.682074
...,...,...,...,...,...
47911,1.795,1.8024,1.784849,1.813500,1.797768
47921,1.780,1.7744,1.766088,1.769486,1.774041
47923,1.780,1.7862,1.775939,1.769486,1.778595
47932,1.773,1.7864,1.773940,1.771125,1.774545


# Modelo 2 Sozinho

In [70]:
def calcular_e_imprimir_metricas_media(resultados_finais):
    metricas_por_fold_modelo = {}

    # Primeiro calcula as métricas por fold e modelo
    for fold, modelos in resultados_finais.items():
        metricas_por_fold_modelo[fold] = {}

        for modelo_nome, df in modelos.items():
            y_true = df["RefractiveIndex_Real"].values
            colunas_pred = [c for c in df.columns if c != "RefractiveIndex_Real"]
            metricas_modelo = {}

            for col_pred in colunas_pred:
                y_pred = df[col_pred].values
                rd = np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100
                r2 = r2_score(y_true, y_pred)
                rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
                rrmse = rmse / np.std(y_true)

                metricas_modelo[col_pred] = {
                    "RD médio": np.float64(rd),
                    "R^2": np.float64(r2),
                    "RMSE": np.float64(rmse),
                    "RRMSE": np.float64(rrmse)
                }
            metricas_por_fold_modelo[fold][modelo_nome] = metricas_modelo

    # Acumuladores para calcular médias e desvios padrão
    medias_modelo_1 = {}
    medias_modelo_2 = {}

    desvios_modelo_1 = {}
    desvios_modelo_2 = {}

    acumulador_1 = {}
    acumulador_2 = {}

    for fold, modelos in metricas_por_fold_modelo.items():
        for modelo_nome, metricas in modelos.items():
            for metodo, metrica in metricas.items():
                if modelo_nome == "modelo_1":
                    acumulador_1.setdefault(metodo, {"RD médio": [], "R^2": [], "RMSE": [], "RRMSE": []})
                    for chave in metrica:
                        acumulador_1[metodo][chave].append(metrica[chave])
                elif modelo_nome == "modelo_2":
                    acumulador_2.setdefault(metodo, {"RD médio": [], "R^2": [], "RMSE": [], "RRMSE": []})
                    for chave in metrica:
                        acumulador_2[metodo][chave].append(metrica[chave])

    # Função para calcular médias e desvios
    def tirar_estatisticas(lista_dicts):
        medias = {k: np.float64(np.mean(v)) for k, v in lista_dicts.items()}
        desvios = {k: np.float64(np.std(v)) for k, v in lista_dicts.items()}
        return medias, desvios

    for metodo, metrica_lista in acumulador_1.items():
        medias_modelo_1[metodo], desvios_modelo_1[metodo] = tirar_estatisticas(metrica_lista)

    for metodo, metrica_lista in acumulador_2.items():
        medias_modelo_2[metodo], desvios_modelo_2[metodo] = tirar_estatisticas(metrica_lista)

    # Impressão formatada
    print("Média e Desvio Padrão das métricas - Modelo 1:")
    for metodo in medias_modelo_1:
        print(f"{metodo}:")
        for chave in medias_modelo_1[metodo]:
            media = medias_modelo_1[metodo][chave]
            desvio = desvios_modelo_1[metodo][chave]
            print(f"  {chave}: média = {media:.4f}, desvio padrão = {desvio:.4f}")

    print("\nMédia e Desvio Padrão das métricas - Modelo 2:")
    for metodo in medias_modelo_2:
        print(f"{metodo}:")
        for chave in medias_modelo_2[metodo]:
            media = medias_modelo_2[metodo][chave]
            desvio = desvios_modelo_2[metodo][chave]
            print(f"  {chave}: média = {media:.4f}, desvio padrão = {desvio:.4f}")

    return medias_modelo_1, medias_modelo_2, desvios_modelo_1, desvios_modelo_2


In [71]:
medias_1, medias_2, desvios_1, desvios_2 = calcular_e_imprimir_metricas_media(resultados_finais)

Média e Desvio Padrão das métricas - Modelo 1:
KNN:
  RD médio: média = 0.8604, desvio padrão = 0.0193
  R^2: média = 0.9600, desvio padrão = 0.0067
  RMSE: média = 0.0370, desvio padrão = 0.0032
  RRMSE: média = 0.1992, desvio padrão = 0.0166
RandomForest:
  RD médio: média = 0.7543, desvio padrão = 0.0187
  R^2: média = 0.9706, desvio padrão = 0.0052
  RMSE: média = 0.0317, desvio padrão = 0.0027
  RRMSE: média = 0.1709, desvio padrão = 0.0149
DecisionTree:
  RD médio: média = 1.1130, desvio padrão = 0.0332
  R^2: média = 0.9416, desvio padrão = 0.0083
  RMSE: média = 0.0447, desvio padrão = 0.0030
  RRMSE: média = 0.2410, desvio padrão = 0.0173
ExtraTrees:
  RD médio: média = 0.6412, desvio padrão = 0.0166
  R^2: média = 0.9740, desvio padrão = 0.0047
  RMSE: média = 0.0298, desvio padrão = 0.0027
  RRMSE: média = 0.1606, desvio padrão = 0.0145

Média e Desvio Padrão das métricas - Modelo 2:
KNN:
  RD médio: média = 0.8556, desvio padrão = 0.0198
  R^2: média = 0.9603, desvio padrão

# Modelo 2 + Regressor -> Agora filtrar das previsoes de modelo 2 os elementos que o classificar knn considera outliers

## Preparando dados Classificador KNN

In [72]:
def gerar_dfs_testes_e_filtrados(df_original, resultados):#Retorna o teste e o train do knn classificador, mas ainda nao tem a coluna label 'esta_em_maiores_erros'
    dicionario_dfs = {}

    for chave_iteracao, dados in resultados.items():
        test_df = dados["test_df"]
        indices_df_metrics = dados["df_metrics"].index
        df_filtrado = df_original.loc[df_original.index.isin(indices_df_metrics)]

        dicionario_dfs[chave_iteracao] = {
            "test_df": test_df,
            "train_df": df_filtrado
        }

    return dicionario_dfs


In [73]:
dfs_train_test_classificator = gerar_dfs_testes_e_filtrados(df, resultados)

In [74]:
def marcar_maiores_erros(dfs_train_test_classificator, indices_acima):#Marca 0 e 1 no df para treino do classificador
    for chave_iteracao, dados in dfs_train_test_classificator.items():
        df_filtrado = dados["train_df"].copy()

        # Garante que a chave de erro corresponda ao padrão no dicionário de erros
        chave_indices_erro = f"{chave_iteracao}"

        # Recupera os índices dos maiores erros dessa iteração
        indices_maiores_erros = indices_acima.get(chave_indices_erro, [])

        # Marca com 1 se o índice está na lista de maiores erros, senão 0
        df_filtrado["maiores_erros"] = df_filtrado.index.isin(indices_maiores_erros).astype(int)

        # Atualiza o dicionário com o DataFrame modificado
        dfs_train_test_classificator[chave_iteracao]["train_df"] = df_filtrado


In [75]:
marcar_maiores_erros(dfs_train_test_classificator, indices_acima)

In [76]:
len(dfs_train_test_classificator['iteracao_fold_0']['train_df'].query("maiores_erros == 1"))


266

In [77]:
dfs_train_test_classificator['iteracao_fold_0']['train_df']

,Li,Be,B,O,Na,Mg,Al,Si,P,K,...,Ta,W,Hg,Tl,Pb,Bi,Th,U,RefractiveIndex,maiores_erros
0,0.0,0.0,0.0,0.630000,0.073333,0.000000,0.000000,0.036667,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.657,0
1,0.0,0.0,0.0,0.624881,0.070428,0.019714,0.000000,0.035214,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.651,0
2,0.0,0.0,0.0,0.620012,0.067698,0.038418,0.000000,0.033849,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.648,0
3,0.0,0.0,0.0,0.615143,0.064971,0.057114,0.000000,0.032485,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.644,0
5,0.0,0.0,0.0,0.615479,0.065113,0.000000,0.000000,0.032557,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.683,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47949,0.0,0.0,0.0,0.614246,0.068289,0.009847,0.063813,0.230730,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.503,0
47950,0.0,0.0,0.0,0.610767,0.080119,0.006285,0.063486,0.229851,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.500,0
47951,0.0,0.0,0.0,0.613590,0.078927,0.006285,0.062479,0.226399,0.006004,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.495,0
47952,0.0,0.0,0.0,0.610737,0.080132,0.006350,0.063369,0.229761,0.000063,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.500,0


## Classificador KNN

In [78]:
def classificar_outliers_knn(dfs_train_test_classificator, n_neighbors=3):
    for chave_iteracao, dados in dfs_train_test_classificator.items():
        df_treino = dados["train_df"]
        df_teste = dados["test_df"].copy()  # cópia para não modificar referência direta

        # Prepara X e y para treino
        X_train = df_treino.drop(columns=["RefractiveIndex", "maiores_erros"])
        y_train = df_treino["maiores_erros"]

        # Prepara X para teste (sem 'RefractiveIndex', sem 'maiores_erros' que ainda não existe nele)
        X_test = df_teste.drop(columns=["RefractiveIndex"])

        # Treina o classificador
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train, y_train)

        # Realiza a previsão
        y_pred = knn.predict(X_test)

        # Salva no df_teste
        df_teste["Outlier"] = y_pred

        # Atualiza o dicionário
        dfs_train_test_classificator[chave_iteracao]["test_df"] = df_teste


In [79]:
dfs_train_test_classificator

{'iteracao_fold_0': {'test_df':              Li   Be         B         O        Na   Mg        Al        Si  \
  4      0.000000  0.0  0.000000  0.702195  0.125045  0.0  0.000000  0.097966   
  38     0.000000  0.0  0.583846  0.673187  0.055152  0.0  0.000000  0.043208   
  49     0.000000  0.0  0.000000  0.723043  0.132569  0.0  0.000000  0.103861   
  71     0.000000  0.0  0.000000  0.786416  0.097540  0.0  0.000000  0.076417   
  75     0.000000  0.0  0.000000  0.712238  0.128641  0.0  0.000000  0.100784   
  ...         ...  ...       ...       ...       ...  ...       ...       ...   
  47911  0.000000  0.0  0.323351  0.650215  0.000000  0.0  0.000000  0.136904   
  47921  0.010496  0.0  0.606815  0.617293  0.000000  0.0  0.006241  0.008994   
  47923  0.000000  0.0  0.622325  0.628947  0.000000  0.0  0.000000  0.002223   
  47932  0.000000  0.0  0.613849  0.621905  0.000000  0.0  0.000000  0.003000   
  47933  0.005456  0.0  0.623077  0.632201  0.000000  0.0  0.000000  0.005294  

In [80]:
classificar_outliers_knn(dfs_train_test_classificator)

In [81]:
for chave_iteracao, dados in dfs_train_test_classificator.items(): 
    df_predito = dados["test_df"]
    n_outliers = (df_predito["Outlier"] == 1).sum()
    print(f"{chave_iteracao}: {n_outliers} outliers preditos")

iteracao_fold_0: 1 outliers preditos
iteracao_fold_1: 3 outliers preditos
iteracao_fold_2: 1 outliers preditos
iteracao_fold_3: 2 outliers preditos
iteracao_fold_4: 3 outliers preditos
iteracao_fold_5: 1 outliers preditos
iteracao_fold_6: 2 outliers preditos
iteracao_fold_7: 0 outliers preditos
iteracao_fold_8: 2 outliers preditos
iteracao_fold_9: 0 outliers preditos


## Preparando dados de treino Regressor

In [82]:
contagem_por_fold = {#Dados usados para treinar o classificador
    chave: len(valores)
    for chave, valores in indices_acima.items()
    if chave.startswith("iteracao_fold_")
}

# Para exibir de forma organizada:
for chave, quantidade in sorted(contagem_por_fold.items()):
    print(f"{chave}: {quantidade} elementos")


iteracao_fold_0: 293 elementos
iteracao_fold_1: 292 elementos
iteracao_fold_2: 292 elementos
iteracao_fold_3: 291 elementos
iteracao_fold_4: 285 elementos
iteracao_fold_5: 300 elementos
iteracao_fold_6: 290 elementos
iteracao_fold_7: 295 elementos
iteracao_fold_8: 290 elementos
iteracao_fold_9: 282 elementos


In [83]:
def gerar_dfs_indices_maiores_erros(df, indices_acima):
    dfs_treino_classificicador = {}

    for chave, indices in indices_acima.items():
        df_filtrado = df.loc[df.index.isin(indices)]
        dfs_treino_classificicador[chave] = df_filtrado

    return dfs_treino_classificicador


In [84]:
dfs_maiores_erros = gerar_dfs_indices_maiores_erros(df, indices_acima)

In [85]:
dfs_maiores_erros["iteracao_fold_0"]

,Li,Be,B,O,Na,Mg,Al,Si,P,K,...,Hf,Ta,W,Hg,Tl,Pb,Bi,Th,U,RefractiveIndex
740,0.000000,0.0,0.021705,0.608880,0.025322,0.000000,0.000539,0.244450,0.00000,0.051190,...,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.540
2223,0.000000,0.0,0.000000,0.589858,0.000000,0.000000,0.000000,0.141274,0.00000,0.076809,...,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.678
2224,0.000000,0.0,0.000000,0.584270,0.000000,0.000000,0.000000,0.127341,0.00000,0.082397,...,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.686
2402,0.000000,0.0,0.000000,0.588426,0.000000,0.000000,0.000000,0.210940,0.00000,0.068174,...,0.0,0.00000,0.000000,0.0,0.0,0.030199,0.000000,0.0,0.0,1.613
2680,0.000000,0.0,0.000000,0.584390,0.000000,0.000000,0.000000,0.168779,0.00000,0.000000,...,0.0,0.00000,0.000000,0.0,0.0,0.246831,0.000000,0.0,0.0,2.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42502,0.009531,0.0,0.158782,0.620533,0.000000,0.000000,0.000000,0.017627,0.00000,0.000000,...,0.0,0.01462,0.027829,0.0,0.0,0.000000,0.000000,0.0,0.0,1.881
42509,0.000000,0.0,0.148750,0.605632,0.000000,0.000000,0.000000,0.019582,0.00000,0.000000,...,0.0,0.00000,0.018460,0.0,0.0,0.000000,0.000000,0.0,0.0,1.880
42781,0.000000,0.0,0.132021,0.629515,0.000000,0.000000,0.000000,0.043616,0.00000,0.000000,...,0.0,0.00000,0.018670,0.0,0.0,0.000000,0.000000,0.0,0.0,1.888
43052,0.000000,0.0,0.011295,0.677804,0.007253,0.000000,0.000000,0.003230,0.08537,0.000000,...,0.0,0.00000,0.016111,0.0,0.0,0.000000,0.090205,0.0,0.0,2.121


In [86]:
for chave_iteracao in dfs_train_test_classificator:
    df_teste = dfs_train_test_classificator[chave_iteracao]["test_df"]
    num_outliers = df_teste.query("Outlier == 1").shape[0]
    print(f"{chave_iteracao}: {num_outliers} outliers")


iteracao_fold_0: 1 outliers
iteracao_fold_1: 3 outliers
iteracao_fold_2: 1 outliers
iteracao_fold_3: 2 outliers
iteracao_fold_4: 3 outliers
iteracao_fold_5: 1 outliers
iteracao_fold_6: 2 outliers
iteracao_fold_7: 0 outliers
iteracao_fold_8: 2 outliers
iteracao_fold_9: 0 outliers


## Regressao sobre teste

In [87]:
from sklearn.neighbors import KNeighborsRegressor

def regredir_maiores_erros_knn(dfs_maiores_erros, dfs_train_test_classificator, n_neighbors=3):
    for chave_iteracao in dfs_maiores_erros.keys():
        # Recupera os DataFrames de treino e teste
        df_treino = dfs_maiores_erros[chave_iteracao]
        df_teste = dfs_train_test_classificator[chave_iteracao]["test_df"]
        
        # Filtra os outliers (onde Outlier == 1)
        df_teste_outliers = df_teste[df_teste["Outlier"] == 1]

        # Verifica se há outliers
        if df_teste_outliers.empty:
            print(f"[{chave_iteracao}] Nenhum outlier encontrado. Iteração ignorada.")
            continue

        # Remove colunas indesejadas
        colunas_excluir = ["RefractiveIndex", "Outlier", "RefractiveIndex_Predito"]
        colunas_X = [col for col in df_treino.columns if col not in colunas_excluir]

        # Prepara os dados
        X_train = df_treino[colunas_X]
        y_train = df_treino["RefractiveIndex"]
        X_test = df_teste_outliers[colunas_X]

        # Ajusta n_neighbors para não ultrapassar número de amostras
        n_samples_fit = len(X_train)
        n_neighbors_ajustado = min(n_neighbors, n_samples_fit)

        if n_neighbors_ajustado == 0:
            print(f"[{chave_iteracao}] Nenhuma amostra de treino disponível. Iteração ignorada.")
            continue

        # Treina o regressor KNN
        knn_regressor = KNeighborsRegressor(n_neighbors=n_neighbors_ajustado)
        knn_regressor.fit(X_train, y_train)

        # Realiza a previsão nos outliers
        y_pred = knn_regressor.predict(X_test)

        # Cria a coluna com o valor predito
        df_teste_outliers = df_teste_outliers.copy()
        df_teste_outliers["RefractiveIndex_Predito"] = y_pred

        # Atualiza o dicionário com os valores preditos
        dfs_maiores_erros[chave_iteracao] = df_teste_outliers


In [88]:
regredir_maiores_erros_knn(dfs_maiores_erros, dfs_train_test_classificator, n_neighbors=3)

[iteracao_fold_7] Nenhum outlier encontrado. Iteração ignorada.
[iteracao_fold_9] Nenhum outlier encontrado. Iteração ignorada.


In [89]:
dfs_maiores_erros["iteracao_fold_0"][["RefractiveIndex", "RefractiveIndex_Predito"]]


,RefractiveIndex,RefractiveIndex_Predito
15073,1.768,1.671333


In [90]:
def substituir_previsoes_modelo_2(resultados_finais, dfs_maiores_erros):
    for chave_iteracao in resultados_finais.keys():
        modelo_2_df = resultados_finais[chave_iteracao]["modelo_2"]
        maiores_erros_df = dfs_maiores_erros.get(chave_iteracao)

        if maiores_erros_df is None:
            print(f"[{chave_iteracao}] Aviso: DataFrame de maiores erros não encontrado.")
            continue

        if "RefractiveIndex_Predito" not in maiores_erros_df.columns:
            print(f"[{chave_iteracao}] Aviso: Coluna 'RefractiveIndex_Predito' ausente.")
            continue

        # Identificar índices comuns entre modelo_2_df e maiores_erros_df
        indices_comuns = modelo_2_df.index.intersection(maiores_erros_df.index)

        if len(indices_comuns) == 0:
            print(f"[{chave_iteracao}] Nenhum índice comum entre modelo_2 e maiores_erros.")
            continue

        # Substituir previsões nos índices comuns
        for idx in indices_comuns:
            valor_predito = maiores_erros_df.at[idx, "RefractiveIndex_Predito"]

            for col in modelo_2_df.columns:
                if col != "RefractiveIndex_Real":
                    modelo_2_df.at[idx, col] = valor_predito

        resultados_finais[chave_iteracao]["modelo_2"] = modelo_2_df


In [91]:
indices_para_mostrar = [9571, 10026, 11331, 14680, 15033, 41038, 41055, 44173, 47213, 47353]
print(resultados_finais['iteracao_fold_0']["modelo_2"].loc[indices_para_mostrar])


       RefractiveIndex_Real     KNN  RandomForest  DecisionTree  ExtraTrees
9571                  1.628  1.6472      1.704745      1.644833    1.691176
10026                 2.175  2.0726      2.068603      2.204000    2.032438
11331                 1.652  1.6428      1.688467      1.650923    1.672977
14680                 1.809  1.8026      1.808640      1.805400    1.809000
15033                 2.170  2.1840      2.196154      2.193500    2.186098
41038                 1.809  1.8094      1.807128      1.803740    1.808223
41055                 1.812  1.8094      1.808921      1.835514    1.811647
44173                 1.720  1.9600      1.922733      1.790000    1.946962
47213                 1.900  2.0070      1.962420      2.008333    2.006609
47353                 2.309  2.3374      2.235062      2.310000    2.286626


In [92]:
substituir_previsoes_modelo_2(resultados_finais, dfs_maiores_erros)


[iteracao_fold_7] Aviso: Coluna 'RefractiveIndex_Predito' ausente.
[iteracao_fold_9] Aviso: Coluna 'RefractiveIndex_Predito' ausente.


In [93]:
indices_para_mostrar = [9571, 10026, 11331, 14680, 47213, 47353]
print(resultados_finais['iteracao_fold_0']["modelo_2"].loc[indices_para_mostrar])


       RefractiveIndex_Real     KNN  RandomForest  DecisionTree  ExtraTrees
9571                  1.628  1.6472      1.704745      1.644833    1.691176
10026                 2.175  2.0726      2.068603      2.204000    2.032438
11331                 1.652  1.6428      1.688467      1.650923    1.672977
14680                 1.809  1.8026      1.808640      1.805400    1.809000
47213                 1.900  2.0070      1.962420      2.008333    2.006609
47353                 2.309  2.3374      2.235062      2.310000    2.286626


In [94]:
def calcular_e_imprimir_metricas_media(resultados_finais):
    metricas_por_fold_modelo = {}

    # Primeiro calcula as métricas por fold e modelo
    for fold, modelos in resultados_finais.items():
        metricas_por_fold_modelo[fold] = {}

        for modelo_nome, df in modelos.items():
            y_true = df["RefractiveIndex_Real"].values
            colunas_pred = [c for c in df.columns if c != "RefractiveIndex_Real"]
            metricas_modelo = {}

            for col_pred in colunas_pred:
                y_pred = df[col_pred].values
                rd = np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100
                r2 = r2_score(y_true, y_pred)
                rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
                rrmse = rmse / np.std(y_true)

                metricas_modelo[col_pred] = {
                    "RD médio": np.float64(rd),
                    "R^2": np.float64(r2),
                    "RMSE": np.float64(rmse),
                    "RRMSE": np.float64(rrmse)
                }
            metricas_por_fold_modelo[fold][modelo_nome] = metricas_modelo

    # Acumuladores para calcular médias e desvios padrão
    medias_modelo_1 = {}
    medias_modelo_2 = {}

    desvios_modelo_1 = {}
    desvios_modelo_2 = {}

    acumulador_1 = {}
    acumulador_2 = {}

    for fold, modelos in metricas_por_fold_modelo.items():
        for modelo_nome, metricas in modelos.items():
            for metodo, metrica in metricas.items():
                if modelo_nome == "modelo_1":
                    acumulador_1.setdefault(metodo, {"RD médio": [], "R^2": [], "RMSE": [], "RRMSE": []})
                    for chave in metrica:
                        acumulador_1[metodo][chave].append(metrica[chave])
                elif modelo_nome == "modelo_2":
                    acumulador_2.setdefault(metodo, {"RD médio": [], "R^2": [], "RMSE": [], "RRMSE": []})
                    for chave in metrica:
                        acumulador_2[metodo][chave].append(metrica[chave])

    # Função para calcular médias e desvios
    def tirar_estatisticas(lista_dicts):
        medias = {k: np.float64(np.mean(v)) for k, v in lista_dicts.items()}
        desvios = {k: np.float64(np.std(v)) for k, v in lista_dicts.items()}
        return medias, desvios

    for metodo, metrica_lista in acumulador_1.items():
        medias_modelo_1[metodo], desvios_modelo_1[metodo] = tirar_estatisticas(metrica_lista)

    for metodo, metrica_lista in acumulador_2.items():
        medias_modelo_2[metodo], desvios_modelo_2[metodo] = tirar_estatisticas(metrica_lista)

    # Impressão formatada
    print("Média e Desvio Padrão das métricas - Modelo 1:")
    for metodo in medias_modelo_1:
        print(f"{metodo}:")
        for chave in medias_modelo_1[metodo]:
            media = medias_modelo_1[metodo][chave]
            desvio = desvios_modelo_1[metodo][chave]
            print(f"  {chave}: média = {media:.4f}, desvio padrão = {desvio:.4f}")

    print("\nMédia e Desvio Padrão das métricas - Modelo 2:")
    for metodo in medias_modelo_2:
        print(f"{metodo}:")
        for chave in medias_modelo_2[metodo]:
            media = medias_modelo_2[metodo][chave]
            desvio = desvios_modelo_2[metodo][chave]
            print(f"  {chave}: média = {media:.4f}, desvio padrão = {desvio:.4f}")

    return medias_modelo_1, medias_modelo_2, desvios_modelo_1, desvios_modelo_2


In [95]:
medias_1, medias_2, desvios_1, desvios_2 = calcular_e_imprimir_metricas_media(resultados_finais)

Média e Desvio Padrão das métricas - Modelo 1:
KNN:
  RD médio: média = 0.8604, desvio padrão = 0.0193
  R^2: média = 0.9600, desvio padrão = 0.0067
  RMSE: média = 0.0370, desvio padrão = 0.0032
  RRMSE: média = 0.1992, desvio padrão = 0.0166
RandomForest:
  RD médio: média = 0.7543, desvio padrão = 0.0187
  R^2: média = 0.9706, desvio padrão = 0.0052
  RMSE: média = 0.0317, desvio padrão = 0.0027
  RRMSE: média = 0.1709, desvio padrão = 0.0149
DecisionTree:
  RD médio: média = 1.1130, desvio padrão = 0.0332
  R^2: média = 0.9416, desvio padrão = 0.0083
  RMSE: média = 0.0447, desvio padrão = 0.0030
  RRMSE: média = 0.2410, desvio padrão = 0.0173
ExtraTrees:
  RD médio: média = 0.6412, desvio padrão = 0.0166
  R^2: média = 0.9740, desvio padrão = 0.0047
  RMSE: média = 0.0298, desvio padrão = 0.0027
  RRMSE: média = 0.1606, desvio padrão = 0.0145

Média e Desvio Padrão das métricas - Modelo 2:
KNN:
  RD médio: média = 0.8585, desvio padrão = 0.0184
  R^2: média = 0.9595, desvio padrão

In [ ]:
import requests

def notificar_telegram(token, chat_id, mensagem):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {"chat_id": chat_id, "text": mensagem}
    response = requests.post(url, data=payload)
    if response.status_code != 200:
        print("Erro ao enviar mensagem:", response.text)

# Exemplo de uso no final do seu script
if __name__ == "__main__":
    # ... seu código principal ...
    
    # Notifique o Telegram no final
    TOKEN = "7657381805:AAH_IsWMKewMROrhpqLCpU9zY8oX45He0MA"
    CHAT_ID = "7178661110"
    MENSAGEM = " -> Abordagem 2 terminou de rodar com sucesso!"

    notificar_telegram(TOKEN, CHAT_ID, MENSAGEM)
